# Check System Design and Simulation functions are behaving nicely

In [ ]:
import os
os.chdir('..') # move to the root directory (from dev)

In [ ]:
import json
import numpy as np

from utils import data_handling
from utils.plotting import init_profile_fig, add_profile
from utils import build_schema, get_Gurobi_WLS_env
from prob_models import prior_model
from energy_system import design_system, evaluate_system

In [ ]:
from experiments.expt_config import *

In [ ]:
# sample scenarios
n_buildings = 3
num_reduced_scenarios = 5
n_samples = 1000
np.random.seed(0)
scenarios = prior_model(n_buildings, n_samples, ids, years)

In [ ]:
# set up Gurobi environment
e = get_Gurobi_WLS_env()
solver_kwargs = {'solver': 'GUROBI', 'env': e}

In [ ]:
# design system
design_results = design_system(
    scenarios,
    dataset_dir,
    building_fname_pattern,
    cost_dict,
    solver_kwargs=solver_kwargs,
    num_reduced_scenarios=num_reduced_scenarios,
    show_progress=True
)

In [ ]:
fig = init_profile_fig(
    y_titles={'y1': 'Building energy (kWh)', 'y2': 'Battery SoC (kWh)'}
    )

fig = add_profile(fig, design_results['e_grids'][0], name='Grid load')
for b in range(n_buildings):
    fig = add_profile(fig, design_results['battery_net_in_flows'][0][b], name=f'{b} battery load')
    fig = add_profile(fig, design_results['SOC'][0][b], name=f'{b} SoC', yaxis='y2')

fig.write_html(f'design_prof_plot.html')

In [ ]:
# print design results
for key in ['objective','objective_contrs','battery_capacities','solar_capacities','grid_con_capacity']:
    print(design_results[key])

In [ ]:
data_handling.save_design_results(design_results, 'test_design_results.csv')

In [ ]:
design_results = data_handling.load_design_results('test_design_results.csv')

In [ ]:
# set up system schema for first scenario
sim_scenario = scenarios[0]
sim_scenario = [[0,2013],[19,2012],[25,2014]]
print(sim_scenario)

building_fname_pattern = 'ly_{id}-{year}.csv'

with open(os.path.join('resources','base_system_params.json')) as jfile:
        base_params = json.load(jfile)

base_params['data_dir_path'] = dataset_dir

# Set system design parameters
base_params['battery_efficiencies'] = [base_params['base_battery_efficiency']]*n_buildings
base_params.pop('base_battery_efficiency', None)
base_params['battery_energy_capacities'] = design_results['battery_capacities'].flatten()
base_params['battery_power_capacities'] = [energy*cost_dict['battery_power_ratio'] for energy in design_results['battery_capacities'].flatten()]
base_params['pv_power_capacities'] = design_results['solar_capacities'].flatten()

# Build schema for each scenario
params = base_params.copy()
params['building_names'] = [f'TB{i}' for i in range(len(sim_scenario))]
params['load_data_paths'] = [building_fname_pattern.format(id=b,year=y) for b,y in sim_scenario]
params['schema_name'] = f'test_EVAL_schema'
schema_path = build_schema(**params)

In [ ]:
# simulate designed system for first scenario
eval_results = evaluate_system(
    schema_path,
    cost_dict,
    design_results['grid_con_capacity'],
    design=True,
    tau=48,
    clip_level='m',
    solver_kwargs={},
    show_progress=True,
    plot=True
)

In [ ]:
print(eval_results)